###### Imports

In [1]:
import time
from datetime import datetime, timedelta
import pandas as pd
import glob
import sys
import numpy as np
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2

[data source](<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/Dataset" property="dct:title" rel="dct:type">Tennis databases, files, and algorithms</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://www.tennisabstract.com/" property="cc:attributionName" rel="cc:attributionURL">Jeff Sackmann / Tennis Abstract</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/JeffSackmann" rel="dct:source">https://github.com/JeffSackmann</a>)

Since the data license is non-commercial, this is a proof-of-concept project and cannot be immediately applied to commercial use.

In [149]:
# read in data
atp_matches = glob.glob('../tennis_atp/atp_matches*.csv')

# separate out by match category
matches_tour = [file for file in atp_matches if 
                       len(file) == len('../tennis_atp/atp_matches_2019.csv')]
matches_qual_chall = glob.glob('../tennis_atp/atp_matches_qual_chall_*.csv')
matches_futures = glob.glob('../tennis_atp/atp_matches_futures_*.csv')
rankings = glob.glob('../tennis_atp/atp_rankings_*.csv')
players = glob.glob('../tennis_atp/atp_players.csv')
matches_amateur = glob.glob('../tennis_atp/atp_matches_amateur.csv')

# singles matches file list
singles_matches = matches_qual_chall + matches_futures + matches_tour + matches_amateur
# Combined DataFrames
singles_matches_df = pd.concat((pd.read_csv(f) for f in singles_matches), ignore_index=True)
matches_glicko2 = singles_matches_df[['tourney_date','winner_id','loser_id','tourney_id']]
# Separate Dataframes
players_df = pd.read_csv(players[0]) 
rankings_df = pd.concat((pd.read_csv(ranks) for ranks in rankings))
matches_qual_chall_df = pd.concat((pd.read_csv(matches) for matches in matches_qual_chall))
matches_futures_df = pd.concat((pd.read_csv(matches) for matches in matches_futures))
matches_tour_df = pd.concat((pd.read_csv(matches) for matches in matches_tour))
matches_amateur_df = pd.read_csv(matches_amateur[0])

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/3051551006.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches_df = pd.concat((pd.read_csv(f) for f in singles_matches), ignore_index=True)
/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/3051551006.py:24: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_amateur_df = pd.read_csv(matches_amateur[0])


Date formatting [source]('https://stackoverflow.com/questions/9750330/how-to-convert-integer-into-date-object-python')

In [152]:
matches_glicko2['tourney_date'] = matches_glicko2['tourney_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/2062871912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_glicko2['tourney_date'] = matches_glicko2['tourney_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))


In [162]:
matches_glicko2.sort_values(by = 'tourney_date', inplace = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/4114524296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_glicko2.sort_values(by = 'tourney_date', inplace = True)


All matches have a winner and loser id, which indicates that all the datapoints can be used for determining the elo ranking.  There is a good amount of standing rank data, suggesting that this data can be used for testing the relative strength of standard ranking vs elo rating in predicting outcomes.

Idea: map real rankings to expected elo

In [4]:
def pct_nan(df):
    '''take in a dataframe, compute the percentage of values that are not 
    missing for each column, and return as a series'''
    pct_nan = pd.Series(df.shape[0] - df.isna().sum().values)/(df.shape[0])
    pct_nan.index = df.isna().sum().index
    return pct_nan.sort_values(ascending = False)

In [5]:
pct_nan(singles_matches_df);

In [ ]:
players_to_instantiate = list(set(players) - set(players_list))

In [158]:
# concept for generating player ids as strings 
# [f'p_{str(player)}' for player in list(m_1900['loser_id'].values)]

In [157]:
# concept for class instantiation
# {player: Player() for player in [f'p_{str(player)}' for player in list(m_1900['loser_id'].values)]}


/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/4114524296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_glicko2.sort_values(by = 'tourney_date', inplace = True)


In [92]:
def epoch(matches, players_list, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.
    
    Future iterations of this function should do away with the p_ , and only should need the 
    players_dict with the keys as integers indicating the player id.
    '''

    players = ['p_' + str(p) for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update attributes for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    for player in players_compete:
        wins = ['p_' + str(winner) for winner in matches[matches['winner_id']==int(player[2:])]['loser_id']]
        losses = ['p_' + str(winner) for winner in matches[matches['loser_id']==int(player[2:])]['winner_id']]
        # get opponents' rating and rd
        # print(f'{wins=}')
        losses_rating = [players_dict[loss].getRating() for loss in losses ]
        losses_rd = [players_dict[loss].getRd() for loss in losses ]
        wins_rating = [players_dict[win].getRating() for win in wins ]
        wins_rd = [players_dict[win].getRd() for win in wins ]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)

    return (players_list,players_dict)
    # opponent_indices = matches[matches['loser_id'].str==player '']

In [211]:
def epochs(match_history,interval_length):
    max_date = max(match_history['tourney_date'])
    min_date = min(match_history['tourney_date'])
    date_range = range(0,(max_date-min_date).days + 1,365)
    players_list = []
    players_dict = {}
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range][1:]
    # iteratively re-update for each epoch
    for cutoff_date in epoch_cutoffs:
        rating_period = match_history[match_history['tourney_date']<=cutoff_date]
        (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)
    

In [217]:
matches_glicko2.tourney_date.index[1000]

852976

In [219]:
matches_glicko2[matches_glicko2.tourney_date < 
                matches_glicko2.tourney_date
                [matches_glicko2.tourney_date.index[1000]]]

,tourney_date,winner_id,loser_id,tourney_id
851995,1877-07-09,113987,114149,1877-540
851977,1877-07-09,113987,114009,1877-540
851978,1877-07-09,114146,113977,1877-540
851979,1877-07-09,114147,114148,1877-540
851980,1877-07-09,114149,114150,1877-540
...,...,...,...,...
852943,1893-07-10,114562,114307,1893-540
852944,1893-07-10,114562,114560,1893-540
852933,1893-07-10,114307,113992,1893-540
852939,1893-07-10,114307,114196,1893-540


In [244]:
def cutDf(df,start,end,column):
    return df[(df[column]>=df[column][df[column].index[start]])&
              (df[column]<=df[column][df[column].index[end]])]


In [230]:
matches_glicko2['tourney_date'][matches_glicko2['tourney_date'].index[1000]]

Timestamp('1893-08-22 00:00:00')

In [230]:
matches_glicko2['tourney_date'][matches_glicko2['tourney_date'].index[1000]]

Timestamp('1893-08-22 00:00:00')

In [ ]:
epochs(cutDf(matches_glicko2,100,208,'tourney_date'),365)

In [ ]:
start = time.time()
epochs(matches_glicko2[matches_glicko2.tourney_date<
                matches_glicko2.tourney_date
                [matches_glicko2.tourney_date.index[100]]],365)
compute_time = time.time()-start
print(compute_time)

In [ ]:
# 23 seconds for first 10_000 matches

The above will updating the ratings and rating deviations for all players 

In [ ]:
list({'a':([1],[2],[1])}.keys())

In [40]:
singles_matches_df = singles_matches_df.sort_values(by = 'tourney_date', ascending = True)

m_1900 = singles_matches_df[singles_matches_df['tourney_date'] <= 19000709]

example_epoch = m_1900
# example_epoch

# example_player = m_1900.loc[m_1900.index[0],'winner_id']
# example_player = eval(f'p_{example_player}')

In [86]:
(updated_list,updated_dict) = epoch(m_1900,[],{})

In [91]:
updated_dict['p_116928'].getRd()

290.31896467295957

In [ ]:
matches_glicko = singles_matches_df[['tourney_date','winner_id','loser_id']]

In [ ]:
example_player.update_player()

In [ ]:
epoch(matches, players_list)

In [ ]:
player_ids

In [ ]:
p_100051.update_player([0],[0],[0])

In [ ]:
p_100051.rating

In [ ]:
p_100051.update_player([0],[0],[0])
p_100051.rating

In [ ]:
def resetPlayer(player):
    exec(f'{player} = Player()')

In [ ]:
resetPlayer('p_100051')

In [ ]:
print(p_100051.rating)
p_100051.update_player([0],[0],[0])
print(p_100051.rating)
resetPlayer('p_100051')
print(p_100051.rating)
p_100051 = Player()
print(p_100051.rating)


In [ ]:
ex

In [ ]:
p_100051.update_player([0],[0],[0])

In [ ]:
p_100051 = Player()

In [ ]:
p_100051.rating

In [ ]:
m_1900 = 

In [ ]:
for player in 

In [ ]:
def epoch()

In [ ]:
for 

In [ ]:
player_ids[1]

In [ ]:
np.unique()

In [ ]:
player_class[len(player_ids)-3]

In [ ]:
player_ids[0]

In [ ]:
p_100001

In [ ]:
players_rated = []
for player in players_df['player_id'].unique():
    exec(f'p_{player} = Player()')

In [ ]:
for player in players_df['player_id'].unique():
    exec(f'p_{str(player)} = Player()')

In [ ]:
exec(f'p_{player} = Player()')

In [ ]:
players_df['player_id'].unique()

In [ ]:
players_rated

In [ ]:
pct_nan(singles_matches_df)[0:25]

efficient solution to creating one df from multiple files is from [Sid](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe) 

In [ ]:
singles_matches_df

In [ ]:
players_rated

In [ ]:
def epoch(matches):
    for match in matches:
        ou
    

In [ ]:
ex

In [ ]:
example_player_wins = example_epoch[example_epoch['winner_id'] == example_player]['loser_id']
example_player_losses = example_epoch[example_epoch['loser_id'] == example_player]['winner_id']
example_player_wins = ['p_' + str(player_id) for player_id in example_player_wins]
example_player_losses = ['p_' + str(player_id) for player_id in example_player_losses]
example_match_ratings = [eval(player).getRating() for player in example_player_wins + example_player_losses]
example_match_rd = [eval(player).rd for player in example_player_wins + example_player_losses]
example_match_outcome = [1 for _ in example_player_wins] + [0 for _ in example_player_losses]

In [ ]:
for 

In [ ]:
example_match_outcome

In [ ]:
example_player.update_player(example_match_ratings,example_match_rd,example_match_outcome)

In [ ]:
example_player.rd

In [ ]:
example_match_outcome

In [ ]:
example_match_ratings

In [ ]:
example_player_wins
example_player_losses

In [ ]:
[]

In [ ]:
m_1900

In [ ]:
singles_matches_df['tourney_date'] = pd.to_datetime(singles_matches_df['tourney_date'], format='%Y%m%d')

In [ ]:
singles_matches_df = singles_matches_df.sort_values(by = 'tourney_date', ascending = True)
singles_matches_df

In [ ]:
pct_nan(singles_matches_df)

In [ ]:
players